In [3]:
import numpy as np
import tables
from datetime import datetime
chunk_number = 400
chunk_length = 16
C3D_input_size = 112
sliding_window_size = 16
sliding_window_stride = 1
number_of_window = 10
number_of_frame_to_load = number_of_window + sliding_window_size-1
resized_stimulus_path = "./resized_stimulus.npy"
extracted_features_path = "./extracted_feature.npy"

In [1]:
%load_ext autoreload
%autoreload 1
%aimport load

In [23]:
def init_file(path):
    with open(path,"w") as myfile:
        myfile.write("")
    myfile.close()
init_file("./test.txt")
with open("./test.txt", "a") as myfile:
    myfile.write("appended text\n")
myfile.close()

In [10]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = 'x'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

printProgressBar(0, 10, prefix = 'Prefix:', suffix = 'Complete', length = 20)
for i in range(10):
    # Do something
    time.sleep(1)
    printProgressBar(i + 1, 10, prefix = 'Prefix:', suffix = 'Complete', length = 20)


Prefix: |xxxxxxxxxxxxxxxxxxxx| 100.0% Complete


In [4]:
#### Load data and take a slice of it ####
# slice size defined by chunk number and chunk length

stimulus_train = load.load_train_stimulus()
print("Stimulus Loaded. Shape:" + str(stimulus_train.shape))
stimulus_train_slice = stimulus_train[:number_of_frame_to_load, :, :, : ]
print("Stimulus sliced. Shape:" + str(stimulus_train_slice.shape))

Stimulus Loaded. Shape:(108000, 3, 128, 128)
Stimulus sliced. Shape:(25, 3, 128, 128)


In [5]:
##### resize images from 128 to 112 and save to file #####
# WARNING: takes time and may crush memory

import skimage.transform as st
resized = np.empty((number_of_frame_to_load,3,C3D_input_size,C3D_input_size))

# Dumb
i = 0
for image in stimulus_train_slice:
    resized[i] = st.resize(image, (3, C3D_input_size, C3D_input_size))
    i+=1

# smarter way
#resize_stimulus_train = st.resize(stimulus_train, (number_of_frame_to_load, 3, 116, 116))
    
print("Data transferred. Shape:" + str(resized.shape))
# Save to file
np.save(resized_stimulus_path, resized)
print("Saved to" + resized_stimulus_path)

Data transferred. Shape:(25, 3, 112, 112)
Saved to./resized_stimulus.npy


In [6]:
#### Load resized images from files ####
resized = np.load(resized_stimulus_path)
print("Loaded. shape" + str(resized.shape))
resized = np.transpose(resized,(0,2,3,1))
print("Tranpsed. shape" + str(resized.shape))

Loaded. shape(25, 3, 112, 112)
Tranpsed. shape(25, 112, 112, 3)


In [9]:
### Chunking using sliding window ###

# resized.shape = (6400, 112, 112, 3)
processed_input = np.zeros((numbers_of_window,sliding_window_size, C3D_input_size, C3D_input_size, 3))

for i in range(number_of_window):
    processed_input[i,:,:,:,:] = resized[i:i+sliding_window_size, :,:,:]

print("Processed Input shape: "+ str(processed_input.shape))

### Checking the result of the sliding window ###
assert processed_input[0,2,:1,:1,:1] == processed_input[1,1,:1,:1,:1]
assert processed_input[0,2,:1,:1,:1] == processed_input[1,1,:1,:1,:1]
print("Test passed.")

Processed Input shape: (10, 16, 112, 112, 3)
Test passed.


In [10]:
### Load pre-trained model and create extractor ###

import h5py
import tensorflow as tf

from keras.models import Model
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.optimizers import SGD
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D

def create_model():
    """ Creates model object with the sequential API:
    https://keras.io/models/sequential/
    """

    model = Sequential()
    input_shape = (16, 112, 112, 3)

    model.add(Conv3D(64, (3, 3, 3), activation='relu',
                     padding='same', name='conv1',
                     input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                           padding='valid', name='pool1'))
    # 2nd layer group
    model.add(Conv3D(128, (3, 3, 3), activation='relu',
                     padding='same', name='conv2'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           padding='valid', name='pool2'))
    # 3rd layer group
    model.add(Conv3D(256, (3, 3, 3), activation='relu',
                     padding='same', name='conv3a'))
    model.add(Conv3D(256, (3, 3, 3), activation='relu',
                     padding='same', name='conv3b'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           padding='valid', name='pool3'))
    # 4th layer group
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                     padding='same', name='conv4a'))
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                     padding='same', name='conv4b'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           padding='valid', name='pool4'))
    # 5th layer group
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                     padding='same', name='conv5a'))
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                     padding='same', name='conv5b'))
    model.add(ZeroPadding3D(padding=((0, 0), (0, 1), (0, 1)), name='zeropad5'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           padding='valid', name='pool5'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(487, activation='softmax', name='fc8'))

    return model

def intermediate_layer_model(layer, model):

 	return Model(inputs=model.input, outputs=model.get_layer(layer).output)

def feature_hook(layer, model, data):

	model = intermediate_layer_model(layer, model)
	return model(data)

def create_features_extractor(model, layer_name):
    extractor = Model(inputs= model.input,
                      outputs= model.get_layer(layer_name).output)
    return extractor


model = create_model()
model.load_weights("c3d-sports1M_weights.h5")
print("Weight loaded")

#model.summary()

output_layer_name = 'fc6'
extractor = create_features_extractor(model,output_layer_name)

Using TensorFlow backend.


Weight loaded
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 1

In [20]:
### Extract feature ###
start_time = datetime.now()
extracted_feature = extractor.predict(processed_input)
end_time = datetime.now()
print("Feature extracted. Shape: "+ str(extracted_feature.shape))
print("Run Time = "+str(end_time - start_time))

Feature extracted. Shape: (10, 4096)
Run Time = 0:00:06.428225


In [49]:
### Chunking WITHOUT sliding window ###

# End result shape should be= (n, 16, 112, 112, 3)
resize_stimulus_train_chunk = np.array_split(resized,chunk_number)
print("Stimulus chunked. " + str(len(resize_stimulus_train_chunk)) + " chunks with shape "+str(resize_stimulus_train_chunk[0].shape))
processed_input = np.zeros((chunk_number,16, 112, 112, 3))
i = 0
for chunk in resize_stimulus_train_chunk:
    processed_input[i,:,:,:,:] = np.transpose(resize_stimulus_train_chunk[0],(0,2,3,1))
    #np.transpose(resize_stimulus_train_chunk[i],(0,2,3,1))
print('Transposed. Shape: '+ str(processed_input.shape))

Stimulus chunked. 400 chunks with shape (16, 3, 112, 112)
Transposed. Shape: (400, 16, 112, 112, 3)
